In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import umap
import tqdm

In [ ]:
news_path = '/content/drive/MyDrive/vk/data.mail.xlsx'


In [ ]:
news_df = pd.read_excel(news_path)


In [ ]:
news_df

,URL,Title,Description,Article Text,Keywords
0,https://kino.mail.ru/news/61905-serial-na-viho...,Сериал на выходные: едкая детективная комедия ...,Рассказываем о густонаселенном сериале «Прести...,"В элитном поселке, где обитают сплошь богачи, ...","СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, ДРАМА, КОМЕДИЯ, ДЕТЕКТИВ"
1,https://kino.mail.ru/news/61913-film-na-vecher...,Фильм на вечер: атмосферный фильм-катастрофа с...,Погружаемся в апокалиптический триллер о франц...,Мир охвачен паникой — повсюду наблюдаются аном...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, ДРАМА, ТРИЛЛЕР, ФАНТАС..."
2,https://kino.mail.ru/news/61900-treiler-dnya-m...,Трейлер дня: Милла Йовович возвращается в трил...,Актриса снялась в новом экшене про пост-апокал...,Вышел дублированный трейлер фантастического тр...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ"
3,https://kino.mail.ru/news/61871-film-na-vecher...,Фильм на вечер: милая зомби-комедия со звездой...,Заряжаемся хулиганским настроением за просмотр...,"Оставшись сиротой, юный Оливер оказывается в т...","СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, КОМЕДИЯ, УЖАСЫ"
4,https://kino.mail.ru/news/61854-treiler-dnya-7...,Трейлер дня: 79-летний Майкл Дуглас играет Бен...,В сериале «Франклин» актер в роли отца-основат...,Вышел трейлер костюмированной 8-серийной драмы...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ"
...,...,...,...,...,...
21088,https://kino.mail.ru/news/16973_novyj_koshmar_...,Новый «Кошмар на улице Вязов» будет очень стра...,Съемки новой версии знаменитого фильма ужасов ...,Съемки новой версии знаменитого фильма ужасов ...,НОВОСТЬ
21089,https://kino.mail.ru/news/16988_razbitaja_gita...,Разбитая гитара Курта Кобейна продана за 100 т...,"Разбитая гитара Курта Кобейна, выставленная на...","Разбитая гитара Курта Кобейна, выставленная на...",НОВОСТЬ
21090,https://kino.mail.ru/news/16977_no_doubt_otloz...,No Doubt отложили выпуск альбома до 2010 года,Возобновившая творческую деятельность группа N...,Возобновившая тв орческую деятельность группа ...,НОВОСТЬ
21091,https://kino.mail.ru/news/20971_po_podozreniju...,По подозрению в убийстве задержан сын Василия ...,Как рассказал источник в правоохранительных ор...,Как рассказал источник в правоохранительных ор...,НОВОСТЬ


In [ ]:

all_sheets = pd.read_excel(news_path, sheet_name=None)

In [ ]:
print("Sheet names:", list(all_sheets.keys()))
news = []
for sheet_name, df in all_sheets.items():
    # if sheet_name == 'deti.mail':
    #   break
    #combined_text = (df['Title'] + '\n' + df['Description'] + '\n' + df['Article Text'])
    combined_text = df['Article Text']
    for text in combined_text:
      news.append(text)


len(news)

Sheet names: ['kino.mail', 'health.mail', 'dom.mail', 'deti.mail']


56721

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(max_features=1000)
# tfidf_matrix = vectorizer.fit_transform(news)

In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

In [ ]:
from sentence_transformers import SentenceTransformer
import tqdm
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
embedder = (SentenceTransformer(model_name))
embedder = embedder.to(device)
# embedding_size = 500
# embedder = tqdm(SentenceTransformer(model_name, embedding_size=embedding_size)
embeddings = embedder.encode(news)

In [ ]:
# test_vector = vectorizer.transform([news[0]])
# test_vector.shape

(1, 1000)

In [ ]:
# vectors = vectorizer.transform(news)
# vectors.shape

(1000, 35761)

In [ ]:
# !pip install umap-learn

In [ ]:
reduced_emb = umap.UMAP(n_components=2, n_neighbors=100, min_dist=0.02).fit_transform(embeddings)

In [ ]:
df = pd.DataFrame()
df['x'] = reduced_emb[:, 0]
df['y'] = reduced_emb[:, 1]
df["text_short"] = news

In [ ]:
# import plotly.express as px
# hover_data = {
#     "text_short": True,
#     "x": False,
#     "y": False
# }
# fig = px.scatter(df, x="x", y="y", template="plotly_dark",
#                    title="Embeddings", hover_data=hover_data)
# fig.update_layout(showlegend=False)
# fig.show()

In [ ]:
!pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 26.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039288 sha256=065731ee8445d7e5ecbc9a7e0c147243414064609452a48e822fe5a6b01f1a1b
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size = 120, min_samples = 7)
labels = clusterer.fit_predict(reduced_emb)
print(f"Num of clusters: {labels.max() + 1}")

Num of clusters: 40


In [ ]:
# number of outliers
df["label"] = [str(label) for label in labels]
num_outliers = len(df[df["label"] == "-1"])
print(f"Num of outliers: {num_outliers} ({num_outliers / len(df) * 100:.2f} % of total)")

Num of outliers: 14046 (24.76 % of total)


In [ ]:
df

,x,y,text_short,label
0,7.274328,-0.330834,элитный посёлок обитать сплошь богач происходи...,-1
1,6.426156,0.137898,мир охватить паника — повсюду наблюдаться аном...,-1
2,7.064607,0.306245,выйти дублировать трейлер фантастический трилл...,-1
3,6.147160,1.727101,остаться сирота юный оливер оказываться тоталь...,-1
4,7.533002,0.090643,выйти трейлер костюмировать 8-серийный драма «...,-1
...,...,...,...,...
995,2.454302,4.190527,скарлатина — инфекционный заболевание вызывать...,-1
996,2.308239,4.114089,вывод прислать исследователь школа медицина бо...,1
997,3.469388,3.716381,заболеть covid-19 нужно следить уровень насыще...,-1
998,4.006565,1.999549,молодой выглядеть жительница английский город ...,-1


In [ ]:
#не заупскать умрет((
# df_no_outliers = df[df["label"] != "-1"]

# # scatter plot
# hover_data = {
#     "text_short": True,
#     "x": False,
#     "y": False
# }
# fig = px.scatter(df_no_outliers, x="x", y="y", template="plotly_dark",
#                    title="Embeddings", color="label", hover_data=hover_data)
# fig.show()

In [ ]:
cluster_labels_df = pd.DataFrame({'Cluster Labels': labels})
cluster_labels_df.to_csv('cluster_labels.csv', index=False)

In [ ]:
# show articles in a specific cluster
cluster = "9"
df_subset = df[df["label"] == cluster].reset_index()
for i,row in df_subset.iterrows():
  print(f"- {row['text_short']}")
  if i == 40:
    break

- «Была очень активная дискуссия вокруг сноса “Соловья” и нового здания, которое планировалось построить на этом месте, с архитектурой, скажем так, очень спорной и выглядящей, мягко говоря, не актуальной. С тех пор мы провели огромную работу с инвестором, поменялась команда архитекторов», — сказал Кузнецов, добавив, что к разработке облика будущего комплекса были приглашены архитекторы Александр Цимайло и Николай Ляшенко (бюро «Цимайло, Ляшенко и партнеры»). По его словам, проект полностью изменился, что позволит избежать критики в его адрес. За 30 лет работы «Соловей» стал любимым местом в Москве для ценителей авторского кино. Посмотрите, как сносили легенду: Ранее Кузнецов сообщал о том, что власти Москвы сумели достичь договоренности с инвестором проекта и на него был выдан новый АГР (свидетельство об утверждении архитектурно-градостроительного решения). Инвестор пообещал сохранить кинозал. Кроме того, документ предполагает расширение общественных пространств в рамках проекта. Киноц

In [ ]:
# !pip install keybert

In [ ]:
from keybert import KeyBERT
cluster = "10"
df_subset = df[df["label"] == cluster].reset_index()
texts_concat = ". ".join(df_subset["text_short"].values)
keywords_and_scores = KeyBERT().extract_keywords(texts_concat,
                                    keyphrase_ngram_range=(1, 1), top_n=10)
print(keywords_and_scores)

[('пропорциями', 0.4874), ('приспособлениями', 0.4855), ('приспособления', 0.4777), ('мероприятия', 0.4767), ('приготовления', 0.4708), ('примирения', 0.4689), ('приложениями', 0.4668), ('пригорания', 0.4662), ('сопротивления', 0.4591), ('переутомления', 0.4571)]


In [ ]:
#тоже гуд работает, но мне кажется скам не скамит
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# dataset, clusters = make_blobs(n_samples = 600, centers = 8, cluster_std = 0.75, random_state = 0)

model = Birch(branching_factor = 50, n_clusters = None, threshold = 1)

model.fit(reduced_emb)

pred = model.predict(reduced_emb)

cluster_labels = model.labels_
cluster_labels.max()


16

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(reduced_emb, cluster_labels)
silhouette_avg

0.3948017

In [ ]:
indices_1 = np.where(cluster_labels == 7)[0]
df_filtered = df[df.index.isin(indices_1)]
# df_filtered
# print(len(df_filtered))
for i in range(len(df_filtered)):
  print(df_filtered.iloc[i]['text_short'])
  if i == 20:
    break


«Мусор» На фоне огромной мусорной горы, возвышающейся над морем и готовой стать его частью, сидит нахмуренный Джереми Айронс. Актеру, обеспокоенному судьбой нашей планеты, предстоит путешествие в разные концы света — от Исландии до Индонезии, а также разговоры с простыми людьми, политиками и учеными. Всех объединяет один печальный факт: их здоровье пострадало от отходов, загрязняющих окружающую среду. Несмотря на в целом удручающую картину, авторы документальной ленты оставляют зрителю надежду — если вопросу переработки отходов будет уделено должное внимание, загрязнение Земли еще можно остановить. «Великий» В начале 1970-х фотожурналист Уильям Юджин Смит делает репортаж об экологической катастрофе, разразившейся в японской деревне Минамата по вине местной фабрики, сливавшей в воду промышленные отходы. Рассказывая историю фотографа, страстно сыгранного Джонни Деппом, режиссер «Великого» Эндрю Левитас кропотливо воссоздает легендарные снимки, запечатлевшие ужасные последствия отравления

In [ ]:
from keybert import KeyBERT
cluster = "7"
# df_subset = df[df["label"] == cluster].reset_index()
texts_concat = ". ".join(df_filtered["text_short"].values)
keywords_and_scores = KeyBERT().extract_keywords(texts_concat,
                                    keyphrase_ngram_range=(1, 1), top_n=5)
print(keywords_and_scores)

[('гастроэнтерологической', 0.5152), ('гистерэктомия', 0.5086), ('гастроэнтерологические', 0.5041), ('гастроэнтерологическим', 0.5027), ('гастроэнтерология', 0.4967)]


In [ ]:
df_filtered.iloc[0]['Description']

'Кухня – притягательное место для ребенка. Еще бы: там столько времени проводит мама, там хранится много вкусного и есть миллион интересных вещей. Впрочем, превращать кухню в игровую комнату – не вариант. А вот привлечь ребенка к приготовлению разных блюд очень даже можно.'

In [ ]:
#по метрике чуть похуже
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
agglomerative = AgglomerativeClustering(n_clusters=20)
cluster_labels = agglomerative.fit_predict(reduced_emb)
cluster_labels

NameError: name 'reduced_emb' is not defined

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(reduced_emb, cluster_labels)
silhouette_avg

0.36019444

In [ ]:
indices_1 = np.where(cluster_labels == 3)[0]
df_filtered = df[df.index.isin(indices_1)]
# df_filtered
# print(len(df_filtered))
for i in range(len(df_filtered)):

  print(df_filtered.iloc[i]['text_short'])
  if i == 20:
    break

Ученые в рамках специального проекта Science of Scare (англ. «Искусство страха») вычислили, какой фильм ужасов объективно является самым страшным. Результаты исследования опубликовало издание MoneySuperMarket. В исследовании участвовало 250 добровольцев, которым показывали фильмы ужасов. К испытуемым был подключен датчик сердцебиения. «Чем выше пульс, тем, значит, быстрее фильм разгоняет кровь в наших зрителях, а это значит, что растут возбуждение и страх как составные части инстинкта «бей или беги», — объясняют авторы проекта. Самый высокий пульс у людей возникал во время просмотра «Синистера» режиссера Скотта Дерриксона. По сюжету, писатель поселяется в доме со страшным прошлым и находит видеозаписи, приоткрывающие тайну произошедшего. Сам он и его близкие после этого также оказываются в опасности. На второй строчке расположился «Астрал: Онлайн» Роба Сэвиджа, на третьей — «Паранормальные явления. Скинамаринк». Также в топ вошли фильмы «Астрал», «Заклятие», «Реинкарнация», «Улыбка», «

In [ ]:
lenta = pd.read_csv('/content/drive/MyDrive/vk/lenta_news_100к.csv')
lenta

,title,text,topic,tags,date,keywords
0,Пятилетний мэр Дорсета проиграл перевыборы,Пятилетний мэр американского городка Дорсет на...,Из жизни,Люди,2014/08/05,штата миннесота перевыборы севере проиграл
1,Бывший советник Трампа усомнился в психическом...,Бывший советник Дональда Трампа Энтони Скараму...,NaN,Политика,2019/08/24,советник энтони бывший трампа дональда
2,Воробьев оценил разработки воспитанников детск...,Губернатор Московской области Андрей Воробьев ...,Россия,Регионы,2017/08/08,мединским воробьев андрей россии посетили вмес...
3,Трамп пообещал американцам новое экономическое...,Президент США Дональд Трамп в своем выступлени...,Экономика,Госэкономика,2017/12/14,президент дональд трамп сша
4,"В ""ЮКОСе"" не поняли смысла действий судебных п...","Ни ""ЮКОС"", ни его дочерние общества никоим обр...",Экономика,Все,2004/07/08,александр дочерние шадрин общества никоим образом
...,...,...,...,...,...,...
99995,Джордж Клуни и Амаль Аламуддин повторно сыграл...,Джордж Клуни и Амаль Клуни (до замужества Алам...,Из жизни,Люди,2014/10/27,в повторно hotel сыграли минувшие свадьбу dane...
99996,Акции «Яндекса» взлетели на 17 процентов на фо...,Акции «Яндекса» в ходе торгов на Московской би...,Бизнес,Бизнес,2017/07/13,бирже чем более выросли московской
99997,Полиция арестовала главу бразильского Google,Бразильские власти 26 сентября арестовали глав...,Интернет и СМИ,Все,2012/09/27,неисполнение главу местного арестовали судебно...
99998,Депутаты в первом чтении разрешили закрывать С...,"Государственная дума России в пятницу, 25 апре...",Интернет и СМИ,Все,2008/04/25,дума закрывать чтении россии законопроект перв...


In [ ]:

filtered_news = lenta[lenta['topic'].isin(['Nan', 'Россия', 'Мир'])]
nan_rows = lenta[lenta['topic'].isna()]
filtered_news = pd.concat([filtered_news, nan_rows])

In [ ]:
filtered_news.topic.unique()

array(['Россия', 'Мир', nan], dtype=object)

In [ ]:
from sentence_transformers import SentenceTransformer
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
embedder = (SentenceTransformer(model_name))
embedder = embedder.to(device)
# embedding_size = 500
# embedder = tqdm(SentenceTransformer(model_name, embedding_size=embedding_size)
embeddings = embedder.encode(news)

In [ ]:
reduced_emb = umap.UMAP(n_components=2, n_neighbors=100, min_dist=0.02).fit_transform(embeddings)

In [ ]:
df = pd.DataFrame()
df['x'] = reduced_emb[:, 0]
df['y'] = reduced_emb[:, 1]
df["text_short"] = news

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size = 69, min_samples = 5)
labels = clusterer.fit_predict(reduced_emb)
print(f"Num of clusters: {labels.max() + 1}")

df["label"] = [str(label) for label in labels]
num_outliers = len(df[df["label"] == "-1"])
print(f"Num of outliers: {num_outliers} ({num_outliers / len(df) * 100:.2f} % of total)")

cluster = "1"
df_subset = df[df["label"] == cluster].reset_index()
for i,row in df_subset.iterrows():
  print(f"- {row['text_short']}")
  if i == 40:
    break

Num of clusters: 5


In [ ]:
# number of outliers
df["label"] = [str(label) for label in labels]
num_outliers = len(df[df["label"] == "-1"])
print(f"Num of outliers: {num_outliers} ({num_outliers / len(df) * 100:.2f} % of total)")

Num of outliers: 206 (2.06 % of total)


-0.0786765

In [ ]:
cluster = "1"
df_subset = df[df["label"] == cluster].reset_index()
for i,row in df_subset.iterrows():
  print(f"- {row['text_short']}")
  if i == 40:
    break

- Ученым из Принстонского университета удалось создать новый тип лазера, сообщается в пресс-релизе, опубликованном на сайте университета. В настоящее время исследователи работают над усовершенствованием своего изобретения, основой для которого стал квантовый каскадный лазер. В современных электронных приборах используются так называемые лазерные диоды, в которых лазерное излучение (световое излучение, почти все фотоны которого имеют одну длину волны и фазу), формируется при подаче электрического тока на полупроводниковый диод. В квантовом каскадном лазере используется похожая схема, только излучающий материал состоит из большого количества полупроводниковых слоев. В качестве излучающего материала исследователи из Принстонского университета использовали много слоев полупроводника толщиной всего в один атом. Во время испытаний ученые обнаружили, что их устройство испускает не один луч, а два с различными длинами волн. При этом у второго луча обнаружились свойства, которые современная тео

In [ ]:
#тоже гуд работает, но мне кажется скам не скамит
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# dataset, clusters = make_blobs(n_samples = 600, centers = 8, cluster_std = 0.75, random_state = 0)

model = Birch(branching_factor = 50, n_clusters = 5, threshold = 1)

model.fit(reduced_emb)

pred = model.predict(reduced_emb)

cluster_labels = model.labels_
cluster_labels.max()


4

In [ ]:
indices_1 = np.where(cluster_labels == 0)[0]
df_filtered = df[df.index.isin(indices_1)]
# df_filtered
# print(len(df_filtered))
for i in range(len(df_filtered)):
  print(df_filtered.iloc[i]['text_short'])
  if i == 20:
    break

Журналист Владимир Познер снял документальный фильм «Шекспир. Предупреждение королям…», премьера которого пройдет 11 декабря в 23:40 на Первом канале. Об этом он рассказал «Ленте.ру». «Вместе со съемочной группой мы сделали фильм о Шекспире. Но не о Шекспире вообще, поскольку это была бы гигантская задача. А о том, как Шекспир относился к власти и к властелинам. Поэтому такое название», — подчеркнул Познер. Он отметил, что в документальной картине присутствуют художественные элементы — небольшие фрагменты из произведений Шекспира. Журналист добавил, что для фильма взял интервью у известных актеров — Кейт Бланшетт, а также Джозефа и Ральфа Файнса. «Снимая такой фильм, нужно очень глубоко влезть в биографию Шескпира, в его произведения. Я, например, выяснил, что Шекспир довольно часто брал уже написанные пьесы и потом просто видоизменял их, переписывал, оставляя нетронутым сам сюжет. Еще я выяснил для себя, насколько он умело критиковал власть во время, когда это было чрезвычайно опасно»

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
embedder = (SentenceTransformer(model_name))

embedder = embedder.to(device)
embeddings = embedder.encode(out_data)

In [ ]:
joblib.dump(embedder, 'embedder_outliners.pkl')


['embedder_outliners.pkl']

['umap_outliners.pkl']

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import Birch

# dataset, clusters = make_blobs(n_samples = 600, centers = 8, cluster_std = 0.75, random_state = 0)

model = Birch(branching_factor = 2, n_clusters = None, threshold = 1)

model.fit(reduced_emb)

pred = model.predict(reduced_emb)

cluster_labels = model.labels_
cluster_labels.max()

12

NameError: name 'news' is not defined

In [ ]:
indices_1 = np.where(cluster_labels == 8)[0]
df_filtered = out_data[out_data.index.isin(indices_1)]
# df_filtered
# print(len(df_filtered))
for i in range(len(df_filtered)):
  print(df_filtered.iloc[i])
  if i == 20:
    break

    #6 - technika
    #8 - деньги

Жительница Чехии сумела незаконно получить прибыль, воспользовавшись лазейками в финансовом законодательстве страны. Она сняла деньги со своего счета, после чего порвала их и частями обменяла на целые банкноты в разных банках. Путем таких манипуляций женщине удалось увеличить первоначальную сумму на
Жительница американского города Проспект, штат Коннектикут, считает, что выиграла в лотерею благодаря совету продавца на автозаправке. Об этом сообщает UPI. Лиллиан Перон (Lillian Perone) покупала билеты моментальной лотереи каждую неделю. Обычно она брала билеты 10X Cash, но в этот раз решила попро
Портрет Мэрилин Монро работы Энди Уорхола из ограниченной серии выставлен на аукцион. По оценкам представителей американского аукционного дома The Potomack Company, стоимость лота может составить от 50 до 100 тысяч долларов, сообщает artdaily.org. Трафаретный отпечаток с лицом актрисы, окрашенным в 
Шон Рэд, один из создателей и президент популярного сервиса знакомств Tinder, продал квартиру на 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
outs = '/content/drive/MyDrive/vk/outliers.csv'
out_data = pd.read_csv(outs).text
out_data = out_data.apply(lambda x: x[:300])
# vectorizer = TfidfVectorizer(max_features = 6000)
# tfidf_matrix = vectorizer.fit_transform(out_data)
# vectors = vectorizer.transform(out_data)

In [ ]:
out_data

0        Министерство иностранных дел РФ предъявило уль...
1        Французский композитор Мишель Легран скончался...
2        Министерство туризма Эквадора в ближайшее воск...
3        Энергетическая компания USEC Inc., которая зан...
4        В Москве прошли похороны тележурналиста и путе...
                               ...                        
12266    Группа скинхедов совершила нападение на пассаж...
12267    Оппозиционеры 18 октября подали новое уведомле...
12268    LG Household & Healthcare, структурное подразд...
12269    В университете немецкого города Веймар был раз...
12270    На юго-востоке Москвы четверо грабителей с топ...
Name: text, Length: 12271, dtype: object

In [ ]:
!pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039299 sha256=2743be0e63dbe899dc36f50773a1d77f4ca323f6e42ad0d64c22fed6eef515e4
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
df = pd.DataFrame()
df['x'] = reduced_emb[:, 0]
df['y'] = reduced_emb[:, 1]
df["text_short"] = out_data

In [ ]:
joblib.dump(clusterer, 'hdbscan_outliners.pkl')

['hdbscan_outliners.pkl']

In [ ]:
import hdbscan
from sentence_transformers import SentenceTransformer
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
embedder = (SentenceTransformer(model_name))

embedder = embedder.to(device)
embeddings = embedder.encode(out_data)
umap_model = umap.UMAP(n_components=2, n_neighbors=100, min_dist=0.02)
reduced_emb = umap_model.fit_transform(embeddings)
# clusterer = hdbscan.HDBSCAN(min_cluster_size = 100, min_samples = 5, prediction_data = True)
clusterer = joblib.load('/content/hdbscan_outliners.pkl')

#labels = clusterer.fit_predict(reduced_emb)
labels = hdbscan.approximate_predict(clusterer, reduced_emb)
print(labels)
print(f"Num of clusters: {labels[0].max() + 1}")

df["label"] = [str(label) for label in labels[0]]
num_outliers = len(df[df["label"] == "-1"])
print(f"Num of outliers: {num_outliers} ({num_outliers / len(df) * 100:.2f} % of total)")


(array([2, 1, 2, ..., 2, 2, 2], dtype=int32), array([1., 1., 1., ..., 1., 1., 1.]))
Num of clusters: 4
Num of outliers: 259 (2.11 % of total)


In [ ]:

cluster = "2"
df_subset = df[df["label"] == cluster].reset_index()
for i,row in df_subset.iterrows():
  print(f"- {row['text_short']}")
  break
  # if i == 40:
  #   break
    #1 - Общественно-политическая активность и события
    #2 - терроризм и преступления
    #0 - события в России

    #0 - События и происшествия
    #1 - утраты
    #2 - Международные отношения
    #3 - судебные процессы

- Министерство иностранных дел РФ предъявило ультиматум Казахстану в ответ на ограничение запусков ракет с Байконура, пишет газета «Известия» 24 января. В том случае, если власти Казахстана не пересмотрят свое решение, Россия грозит прекратить сотрудничество по всем совместным проектам в области запус


In [ ]:
import pickle
dic = {0: 'События и происшетсвия', 1: 'Утраты', 2: 'Международные отношения', 3: "Cудебные процессы"}
pickle_file_path = 'match.pickle'
with open(pickle_file_path, 'wb') as pickle_file:
    # Dump the dictionary to the pickle file
    pickle.dump(dic, pickle_file)

In [ ]:
import joblib

['outliners.pkl']

In [ ]:
df

,x,y,text_short,label
0,3.797441,6.466629,Министерство иностранных дел РФ предъявило уль...,1
1,2.997145,1.601501,Французский композитор Мишель Легран скончался...,1
2,2.377939,5.824254,Министерство туризма Эквадора в ближайшее воск...,1
3,4.845646,4.734970,"Энергетическая компания USEC Inc., которая зан...",1
4,3.085497,1.775823,В Москве прошли похороны тележурналиста и путе...,1
...,...,...,...,...
12266,3.289481,2.925387,Группа скинхедов совершила нападение на пассаж...,1
12267,4.124057,5.282886,Оппозиционеры 18 октября подали новое уведомле...,1
12268,1.895277,6.992617,"LG Household & Healthcare, структурное подразд...",1
12269,2.352939,6.917259,В университете немецкого города Веймар был раз...,1


In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='sk-8l3dfizD4UPDqKwEXuOkzVXhEjnsJIgL', base_url="https://api.proxyapi.ru/openai/v1")


def get_tag(texts):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Тебе дадут несколько новостей, тебе нужно понять что в них общее, написать их общую тему, то есть о чем они. Ты должен описать эту тему один или двуми словами"
            },
            {
                "role": "user",
                "content": texts

            }
        ],
        temperature=0.7,
        max_tokens=64,
        top_p=1
    )
    return response.choices[0].message.content


claster_names = dict()
for i in range(labels.max()):
    texts = ''
    df_subset = df[df["label"] == str(i)].reset_index()
    for j,row in df_subset.iterrows():
        texts += '/n' + row['text_short'][:100]
        if j == 10:
          continue
    tag = get_tag(texts)
    claster_names[i] = tag

APIStatusError: Error code: 402 - {'detail': 'Insufficient balance to run this request.'}

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
!pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
